# figures galore

results visualizations for paper or something idk

## setup, data loading needed for most visualizations

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os
import sys
module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
import json
import math
import os
from pathlib import Path
from typing import List
import sys

import cartopy
import cartopy.crs as ccrs
import cv2
import dataclass_array as dca
import jax.numpy as jnp
import matplotlib as mpl
import matplotlib.pyplot as plt
import mitsuba as mi
import numpy as np
import open3d as o3d
from PIL import Image
import pandas as pd
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
import pycolmap
import pyransac3d as pyrsc
import pyrender
import quaternion
import scipy.interpolate
import trimesh
import visu3d as v3d
import yaml

sys.path.append(os.path.abspath(os.path.join("..", "bop_toolkit")))
from bop_toolkit.bop_toolkit_lib.pose_error import vsd, mssd, mspd
from bop_toolkit.bop_toolkit_lib.misc import get_symmetry_transformations
from bop_toolkit.bop_toolkit_lib.renderer import create_renderer

import burybarrel.colmap_util as cutil
from burybarrel.image import render_v3d, render_models, to_contour, overlay_img_alpha
from burybarrel.camera import RadialCamera, load_v3dcams
from burybarrel.mesh import subdivide_mesh, recolor_mesh, segment_pc_from_mask, segment_pc_from_masks
from burybarrel.transform import T_from_blender, T_from_translation, scale_T_translation, scale_cams, scale_pc, icp, qangle, qmean, closest_quat_sym, get_axes_rot, apply_T_xyz
from burybarrel.plotting import get_line3d_trace, get_axes_traces, get_plane_zup, get_surface_line_traces, get_ray_trace, generate_domain, get_carree_axis, get_carree_gl, get_trimesh_traces
from burybarrel.utils import rgb2hex, cmapvals, match_lists, invert_idxs
from burybarrel.estimators import ransac
from burybarrel.foundpose_fit import fitcams, camloss, camcost, qcost, qloss

In [ ]:
# tmpdir = Path("/scratch/jeyan/barreldata/tmp")
tmpdir = Path("/Users/jerry/Projects/ms-stuff/barrel-playground/tmp")
# metricscsvpath = Path("/scratch/jeyan/barreldata/results/allmetrics.csv")
metricscsvpath = Path("/Users/jerry/Projects/ms-stuff/barrel-playground/barrels/results/allmetrics.csv")
metricsdf = pd.read_csv(metricscsvpath)
metricsdf.head()

In [ ]:
metricsdf[["avg_vsd", "avg_mssd", "avg_mspd", "burial_error_depth", "multiview_fitted", "pose_type", "use_icp"]].groupby(["multiview_fitted", "use_icp", "pose_type"]).mean()

In [ ]:
def filtermetrics(df, name=None, multiview_fitted=None, use_icp=None, pose_type=None):
    if multiview_fitted is not None:
        df = df[df["multiview_fitted"] == multiview_fitted]
    if use_icp is not None:
        df = df[df["use_icp"] == use_icp]
    if pose_type is not None:
        df = df[df["pose_type"] == pose_type]
    if name is not None:
        df = df[df["dataset"] == name]
    return df

In [ ]:
nsamples = len(filtermetrics(metricsdf, multiview_fitted=True, use_icp=True, pose_type="coarse"))
print(f"Number of labeled samples: {nsamples}")

In [ ]:
# loading EVERYTHING
datasetname = "barrel4"
singledatarows = metricsdf[metricsdf["dataset"] == datasetname]

# multiview fit, coarse, icp
resrow = singledatarows[singledatarows["multiview_fitted"] & (singledatarows["pose_type"] == "coarse") & singledatarows["use_icp"]].iloc[0]
datadir = Path(resrow["datadir"])
datadir = Path("/Users/jerry/Projects/ms-stuff/barrel-playground/barrels/data/input_data") / datadir.name
resdir = Path(resrow["resdir"])
resdir = Path("/Users/jerry/Projects/ms-stuff/barrel-playground/barrels/results") / Path(*resdir.parts[-3:])
object_name = resrow["object_name"]

with open("../configs/blender_gt_info.yaml", "rt") as f:
    gtinfo = yaml.safe_load(f)
with open(datadir / "gt_obj2cam.json", "rt") as f:
    gtposes = yaml.safe_load(f)
with open(datadir / "info.json", "rt") as f:
    datainfo = yaml.safe_load(f)
with open(datadir / "camera.json", "rt") as f:
    caminfo = yaml.safe_load(f)
with open(resdir / "../../sam-masks/masksinfo.json", "rt") as f:
    masksinfo = yaml.safe_load(f)
posepath = resdir / "estimated-poses.json"
estinfopath = resdir / "reconstruction-info.json"
with open(posepath, "rt") as f:
    ests = yaml.safe_load(f)
with open(estinfopath, "rt") as f:
    estinfo = yaml.safe_load(f)
with open(resdir / "../../foundpose-output/inference/estimated-poses.json", "rt") as f:
    foundposeests = yaml.safe_load(f)
cams, imgpaths = load_v3dcams(resdir / "../../colmap-out/cam_poses.json", img_parent=datadir / "rgb")
names = [imgpath.stem for imgpath in imgpaths]

densepath = resdir / "../../openmvs-out/scene_dense_trimeshvalid.ply"
densetrimesh: trimesh.PointCloud = trimesh.load_mesh(densepath)
densepc = v3d.Point3d(p=densetrimesh.vertices, rgb=densetrimesh.colors[:, :3])

camspec = RadialCamera.from_jsonargs(**caminfo)
# objdir = Path("/scratch/jeyan/barreldata/models3d")
objdir = Path("/Users/jerry/Projects/ms-stuff/barrel-playground/models3d")
objpath = objdir / object_name
with open(objdir / "model_info.json", "rt") as f:
    objinfo = yaml.safe_load(f)
mesh: trimesh.Trimesh = trimesh.load_mesh(objpath)
meshpc = v3d.Point3d(p=mesh.vertices, rgb=[255, 0, 0])

## some 3d overlay wireframe visualization thing

In [ ]:
estidx = 5
gtidx = estidx
camidx = estidx
# note these might be inconsistent indices, but it would depend on SAM failing
est = ests[estidx]
gt = gtposes[gtidx]
colmapcam = cams[camidx]
singlemasksinfo = next(filter(lambda x: Path(x["img_path"]).stem == Path(est["img_path"]).stem, masksinfo))
estscale = estinfo["scalefactor"]
reconstr_mesh: trimesh.Trimesh = trimesh.load(resdir / "../../openmvs-out/scene_dense_mesh_refine.ply")
# reconstr_mesh.vertices = reconstr_mesh.vertices * estscale
bestidx = singlemasksinfo["best_idx"]
mask = np.array(Image.open(Path("/Users/jerry/Projects/ms-stuff/barrel-playground/barrels/results") / Path(*Path(singlemasksinfo["mask_paths"][bestidx]).parts[-4:])).convert("L"))
# est, gt, cam, mask
est["img_path"], gt["img_path"], imgpaths[camidx], singlemasksinfo["img_path"]

In [ ]:
# meshvertmask = np.zeros(reconstr_mesh.vertices.shape[0], dtype=bool)
# flooridxsmesh = invert_idxs(segment_pc_from_mask(reconstr_mesh.vertices, mask, colmapcam), reconstr_mesh.vertices.shape[0])
# meshvertmask[flooridxsmesh] = True
# reconstr_floor_mesh = reconstr_mesh.copy()
# # reconstr_floor_mesh.update_vertices(meshvertmask)
# face_mask = meshvertmask[reconstr_mesh.faces].all(axis=1)
# reconstr_floor_mesh.update_faces(face_mask)

In [ ]:
meshcol = np.array([255, 0, 0, 255], dtype=np.uint8)
planecol = np.array([0, 0, 255, 255], dtype=np.uint8)
black = np.array([0, 0, 0, 255], dtype=np.uint8)
transparent = np.array([0, 0, 0, 0], dtype=np.uint8)
mesh = recolor_mesh(mesh, meshcol)
plane = trimesh.creation.box(extents=(100, 100, 0.01))
plane = subdivide_mesh(plane, 7)
plane = recolor_mesh(plane, planecol)

In [ ]:
obj2cam = v3d.Transform(R=est["R"], t=est["t"])
floor2cam = v3d.Transform(R=est["R_floor"], t=est["t_floor"])
obj2camgt = v3d.Transform(R=gt["R"], t=gt["t"])
floor2camgt = v3d.Transform(R=gt["R_floor"], t=gt["t_floor"])

In [ ]:
cam = v3d.Camera(spec=camspec, world_from_cam=v3d.Transform.identity())

In [ ]:
# [xmin, ymin, width, height]
crop = [0, 120, 1920, 875]
pilcrop = [crop[0], crop[1], crop[0] + crop[2], crop[1] + crop[3]]
bestidx = singlemasksinfo["best_idx"]
maskalpha = cv2.cvtColor(mask, cv2.COLOR_GRAY2RGBA)
maskalpha[~(maskalpha[..., 0] > 0), 3] = 0

# predicted code
maskrgb, _, _ = render_models(cam, [mesh, plane], [obj2cam, floor2cam], light_intensity=1000, flags=pyrender.RenderFlags.FLAT)
objmask = np.all(maskrgb == [255, 0, 0], axis=-1)
rgb_wireframe, _, _ = render_models(cam, [recolor_mesh(plane, black)], [floor2cam], wireframe=True, flags=pyrender.RenderFlags.RGBA | pyrender.RenderFlags.FLAT)
rgb_wireframe = rgb_wireframe * ~objmask[..., None]
rgb_wireframe[rgb_wireframe[..., 3] > 0, 3] = 255
rgb_primoverlay, _, _ = render_models(cam, [mesh, plane], [obj2cam, floor2cam], light_intensity=50, wireframe=False)
# Image.fromarray(rgb_wireframe).save(tmpdir / "rendered_wireframe.png")
# Image.fromarray(rgb_primoverlay).save(tmpdir / "rendered.png")
# img = cv2.cvtColor(cv2.imread(est["img_path"]), cv2.COLOR_BGR2RGB)
img = cv2.cvtColor(cv2.imread(str(datadir / Path(*Path(est["img_path"]).parts[-2:]))), cv2.COLOR_BGR2RGB)
if img.shape[0] == 1080 and img.shape[1] == 1920:
    Image.fromarray(overlay_img_alpha(img, maskalpha, alpha=0.3)).crop(pilcrop).save(tmpdir / f"rgbres_{datasetname}.png")
else:
    Image.fromarray(overlay_img_alpha(img, maskalpha, alpha=0.3)).save(tmpdir / f"rgbres_{datasetname}.png")
alpha = 0.6
img = cv2.addWeighted(img, alpha, rgb_primoverlay, 1 - alpha, 0)
# wireframewhere = np.where(rgb_wireframe[..., 3] > 0)
# img[wireframewhere] = rgb_wireframe[wireframewhere][..., :3]
img = overlay_img_alpha(img, rgb_wireframe)
mesh_wireframe, _, _ = render_models(cam, [recolor_mesh(mesh, black)], [obj2cam], wireframe=True, flags=pyrender.RenderFlags.RGBA | pyrender.RenderFlags.FLAT)
mesh_wireframe[mesh_wireframe[..., 3] > 0, 3] = 255
img = overlay_img_alpha(img, mesh_wireframe, alpha=0.3)
if img.shape[0] == 1080 and img.shape[1] == 1920:
    Image.fromarray(img).crop(pilcrop).save(tmpdir / f"rendered_overlay_{datasetname}.png")
else:
    Image.fromarray(img).save(tmpdir / f"rendered_overlay_{datasetname}.png")

# gt code
maskrgb, _, _ = render_models(cam, [mesh, plane], [obj2camgt, floor2camgt], light_intensity=1000, flags=pyrender.RenderFlags.FLAT)
objmask = np.all(maskrgb == [255, 0, 0], axis=-1)
rgb_wireframe, _, _ = render_models(cam, [recolor_mesh(plane, black)], [floor2camgt], wireframe=True, flags=pyrender.RenderFlags.RGBA | pyrender.RenderFlags.FLAT)
rgb_wireframe = rgb_wireframe * ~objmask[..., None]
rgb_wireframe[rgb_wireframe[..., 3] > 0, 3] = 255
rgb_primoverlay, _, _ = render_models(cam, [mesh, plane], [obj2camgt, floor2camgt], light_intensity=50, wireframe=False)
# Image.fromarray(rgb_wireframe).save(tmpdir / "rendered_wireframe_gt.png")
# Image.fromarray(rgb_primoverlay).save(tmpdir / "rendered_gt.png")
# img = cv2.cvtColor(cv2.imread(est["img_path"]), cv2.COLOR_BGR2RGB)
img = cv2.cvtColor(cv2.imread(str(datadir / Path(*Path(est["img_path"]).parts[-2:]))), cv2.COLOR_BGR2RGB)
alpha = 0.6
img = cv2.addWeighted(img, alpha, rgb_primoverlay, 1 - alpha, 0)
# wireframewhere = np.where(rgb_wireframe[..., 3] > 0)
# img[wireframewhere] = rgb_wireframe[wireframewhere][..., :3]
img = overlay_img_alpha(img, rgb_wireframe)
mesh_wireframe, _, _ = render_models(cam, [recolor_mesh(mesh, black)], [obj2camgt], wireframe=True, flags=pyrender.RenderFlags.RGBA | pyrender.RenderFlags.FLAT)
mesh_wireframe[mesh_wireframe[..., 3] > 0, 3] = 255
img = overlay_img_alpha(img, mesh_wireframe, alpha=0.3)
if img.shape[0] == 1080 and img.shape[1] == 1920:
    Image.fromarray(img).crop(pilcrop).save(tmpdir / f"rendered_overlay_{datasetname}_gt.png")
else:
    Image.fromarray(img).save(tmpdir / f"rendered_overlay_{datasetname}_gt.png")

## visualizing mask

In [ ]:
idx = 1
singlemasksinfo = masksinfo[idx]
singlemasksinfo

In [ ]:
bestidx = singlemasksinfo["best_idx"]
# [x_min, y_min, x_max, y_max]
bbox = list(map(int, singlemasksinfo["bboxes"][bestidx]))
mask = np.array(Image.open(singlemasksinfo["mask_paths"][bestidx]).convert("L"))
maskbool = mask > 0
img = np.array(Image.open(singlemasksinfo["img_path"]).convert("RGBA"))

In [ ]:
maskalpha = np.zeros((*img.shape[:2], 4), dtype=np.uint8)
maskalpha[maskbool, 0:3] = [255, 0, 0]
maskalpha[maskbool, 3] = 255
maskalpha[~maskbool] = img[~maskbool]
alpha = 0.5
overlayed = cv2.addWeighted(img, alpha, maskalpha, 1 - alpha, 0)
overlayed = cv2.rectangle(overlayed, (bbox[0], bbox[1]), (bbox[2], bbox[3]), (255, 197, 71, 255), 10)
Image.fromarray(overlayed).save(tmpdir / "rendered_overlay_mask.png")

## 3d reconstruction + multiview visualization thing + scale ambiguity visualization

In [ ]:
# Tworld_obj = T_from_blender(gtinfo[datasetname]["R"], gtinfo[datasetname]["t"], scale=gtinfo[datasetname]["scalefactor"])
scale = 1 / gtinfo[datasetname]["scalefactor"]
T_zup = T_from_blender(gtinfo[datasetname]["R_floor"], gtinfo[datasetname]["t_floor"], scale=1).inv

In [ ]:
datainfo

In [ ]:
gtposes[0]

In [ ]:
densepc_zup = scale_pc(T_zup @ densepc, scale)
cams_zup = scale_cams(cams.apply_transform(T_zup), scale)
cam1 = cams_zup[0]
cam2 = cams_zup[-1]
foundposeest1 = list(filter(lambda x: Path(x["img_path"]).stem == imgpaths[0].stem, foundposeests))[1]
foundposeT1 = v3d.Transform(R=foundposeest1["R"], t=foundposeest1["t"])
T_objworld1 = cam1.world_from_cam @ foundposeT1
foundposeest2 = list(filter(lambda x: Path(x["img_path"]).stem == imgpaths[-1].stem, foundposeests))[4]
foundposeT2 = v3d.Transform(R=foundposeest2["R"], t=foundposeest2["t"])
T_objworld2 = cam2.world_from_cam @ foundposeT2
# v3d.make_fig([densepc_zup, cam1, cam2, get_line3d_trace(cams_zup.world_from_cam.t, markersize=3, linewidth=3), T_objworld @ meshpc])
_, wiretrace1 = get_trimesh_traces(mesh.copy().apply_transform(T_objworld1.matrix4x4), wirecolor="rgba(255, 0, 0, 150)")
_, wiretrace2 = get_trimesh_traces(mesh.copy().apply_transform(T_objworld2.matrix4x4), wirecolor="rgba(0, 0, 255, 150)")
fig = v3d.make_fig([densepc_zup, cam1, cam2.make_traces()[0].update(marker={"color": "rgb(0, 0, 255)"}), get_line3d_trace(cams_zup.world_from_cam.t, markersize=3, linewidth=3), wiretrace1, wiretrace2])
fig.update_layout(
    showlegend=False,
    margin={'t':0,'l':0,'b':0,'r':0},
)
config = {
    "toImageButtonOptions": {
        "format": "png", # one of png, svg, jpeg, webp
        "width": 1200,
        "height": 900,
        "filename": "multiview3d-2cams",
        # 'scale': 10,
    }
}
fig.update_scenes(xaxis_visible=False, yaxis_visible=False, zaxis_visible=False)
fig.show(config=config)

# /scratch/jeyan/barreldata/tmp/newplot.png
# /Users/jerry/Projects/ms-stuff/barrel-playground/tmp/newplot.png

In [ ]:
img = np.array(Image.open(imgpaths[0]).convert("RGBA"))
objwireframe, _, _ = render_models(cam1, [recolor_mesh(mesh, [255, 0, 0, 255]), recolor_mesh(mesh, [0, 0, 255, 255])], [T_objworld1, T_objworld2], light_intensity=5000, wireframe=True, flags=pyrender.RenderFlags.RGBA)
wireframemask = objwireframe[..., 3] > 0
objwireframe[wireframemask, 3] = 255
# objwireframe[~wireframemask] = img[~wireframemask]
Image.fromarray(overlay_img_alpha(img, objwireframe, alpha=0.5)).save(tmpdir / "cam1_wireframe.png")

In [ ]:
img = np.array(Image.open(imgpaths[-1]).convert("RGBA"))
objwireframe, _, _ = render_models(cam2, [recolor_mesh(mesh, [255, 0, 0, 255]), recolor_mesh(mesh, [0, 0, 255, 255])], [T_objworld1, T_objworld2], light_intensity=5000, wireframe=True, flags=pyrender.RenderFlags.RGBA)
wireframemask = objwireframe[..., 3] > 0
objwireframe[wireframemask, 3] = 255
# objwireframe[~wireframemask] = img[~wireframemask]
Image.fromarray(overlay_img_alpha(img, objwireframe, alpha=0.5)).save(tmpdir / "cam2_wireframe.png")

In [ ]:
densepc_zup_unscaled = T_zup @ densepc
cams_zup_unscaled = cams.apply_transform(T_zup)
fig = v3d.make_fig([densepc_zup_unscaled, cams_zup_unscaled])
plotlycam = {
    'center': {'x': 0, 'y': 0, 'z': 0},
    'eye': {'x': 0.979302700454209, 'y': -0.8701522198840933, 'z': 0.7050176661837763},
    'projection': {'type': 'perspective'},
    'up': {'x': 0, 'y': 0, 'z': 1},
}
fig.update_layout(
    scene_camera=plotlycam,
    showlegend=False,
    margin={'t':0,'l':0,'b':0,'r':0},
    # template='none',
    scene=dict(
        xaxis=dict(
            # showgrid=False,
            # zeroline=False,
            showticklabels=False
        ),
        yaxis=dict(
            # showgrid=False,
            # zeroline=False,
            showticklabels=False
        ),
        zaxis=dict(
            # showgrid=False,
            # zeroline=False,
            showticklabels=False
        ),
    ),
)
fig.update_scenes(xaxis_visible=False, yaxis_visible=True, zaxis_visible=False)
config = {
    "toImageButtonOptions": {
        "format": "png", # one of png, svg, jpeg, webp
        "filename": "colmapraw",
        "width": None,
        "height": None,
        "scale": 1,
    }
}
fig.write_image(tmpdir / "colmapraw.png", scale=1)
f = go.FigureWidget(fig)
f
# fig.show(config=config)

In [ ]:
f.layout['scene']['camera']

In [ ]:
obj2cams_gt = []
for i, gtposeinfo in enumerate(sorted(gtposes, key=lambda x: x["img_path"])):
    T = v3d.Transform(R=gtposeinfo["R"], t=gtposeinfo["t"])
    obj2cams_gt.append(T)
obj2cams_gt = dca.stack(obj2cams_gt)
obj2cams_est = []
for i, estposeinfo in enumerate(filter(lambda x: x["hypothesis_id"] == "0", sorted(ests, key=lambda x: x["img_path"]))):
    T = v3d.Transform(R=estposeinfo["R"], t=estposeinfo["t"])
    obj2cams_est.append(T)
obj2cams_est = dca.stack(obj2cams_est)

In [ ]:
mesh_camfront_unscaledpc = (cams_zup_unscaled.world_from_cam @ obj2cams_est @ meshpc[None, ...])
colors = (255 * cmapvals(np.arange(len(cams_zup_unscaled)), cmap="tab20")).astype(np.uint8)
traces = []
skip = 4
for i, cam_zup_unscaled in enumerate(cams_zup_unscaled):
    if i % skip != 0:
        continue
    camtrace = cam_zup_unscaled.make_traces()[0].update(line={"color": rgb2hex(colors[i])})
    # meshtrace = mesh_camfront_unscaledpc[i].replace(rgb=colors[i])
    _, wiretrace = get_trimesh_traces(mesh.copy().apply_transform((cams_zup_unscaled.world_from_cam[i] @ obj2cams_est[i]).matrix4x4), wirecolor=rgb2hex(colors[i]))
    traces.append(camtrace)
    traces.append(wiretrace)
fig = v3d.make_fig([*traces, densepc_zup_unscaled, get_line3d_trace(cams_zup_unscaled.world_from_cam.t, markersize=1, linewidth=2, linecolor="#ab63fa")])
plotlycam = {
    'center': {'x': 0, 'y': 0, 'z': 0},
    'eye': {'x': 1.7135838058633417, 'y': -1.2023497067709676, 'z': 0.5527076287791509},
    'projection': {'type': 'perspective'},
    'up': {'x': 0, 'y': 0, 'z': 1},
}
fig.update_layout(
    scene_camera=plotlycam,
    showlegend=False,
    margin={'t':0,'l':0,'b':0,'r':0},
)
fig.write_image(tmpdir / "colmap-unscaled.png", scale=2)
f = go.FigureWidget(fig)
f

In [ ]:
f.layout['scene']['camera']

In [ ]:
mesh_camfront_pc = (cams_zup.world_from_cam @ obj2cams_est @ meshpc[None, ...])
traces = []
for i, cam_zup in enumerate(cams_zup):
    if i % skip != 0:
        continue
    camtrace = cam_zup.make_traces()[0].update(line={"color": rgb2hex(colors[i])})
    # meshtrace = mesh_camfront_pc[i].replace(rgb=colors[i])
    _, wiretrace = get_trimesh_traces(mesh.copy().apply_transform((cams_zup.world_from_cam[i] @ obj2cams_est[i]).matrix4x4), wirecolor=rgb2hex(colors[i]))
    traces.append(camtrace)
    traces.append(wiretrace)
fig = v3d.make_fig([*traces, densepc_zup, get_line3d_trace(cams_zup.world_from_cam.t, markersize=3, linewidth=3, linecolor="#ab63fa")])
plotlycam = {
    'center': {'x': 0, 'y': 0, 'z': 0},
    'eye': {'x': 1.7135838058633417, 'y': -1.2023497067709676, 'z': 0.5527076287791509},
    'projection': {'type': 'perspective'},
    'up': {'x': 0, 'y': 0, 'z': 1},
}
fig.update_layout(
    scene_camera=plotlycam,
    showlegend=False,
    margin={'t':0,'l':0,'b':0,'r':0},
)
fig.write_image(tmpdir / "colmap-scaled.png", scale=2)
f = go.FigureWidget(fig)
f

In [ ]:
f.layout['scene']['camera']


## ICP + rotation visualization

In [ ]:
objinfo["barrelsingle-scaled.ply"]

In [ ]:
cameras = cams
use_coarse = True
scenepts = densepc.replace()
masks = []
for i, maskinfo in enumerate(masksinfo):
    maskpath = Path(maskinfo["mask_paths"][maskinfo["best_idx"]])
    maskpath = Path("/Users/jerry/Projects/ms-stuff/barrel-playground/barrels/results") / Path(*maskpath.parts[-4:])
    masks.append(np.array(Image.open(maskpath).convert("L")))
masks = np.array(masks)
objectmesh = mesh.copy()
objectsymmetries = get_symmetry_transformations(objinfo["barrelsingle-scaled.ply"], 0.01)


cams = cameras
# names, cameras are already assumed to be ordered and filtered
# depending on failed registration in COLMAP or SAM masks
name2cam = {name: cam for name, cam in zip(names, cameras)}
# foundpose results are used as reference for image ids
name2imgid = {}
name2imgpath = {}
obj2cams: v3d.Transform = []
# camera for each hypothesis. If multiple hypotheses for each image, there will be
# duplicate cameras in here
camhyps: v3d.Camera = []
# colmap usually can't reconstruct every camera pose, so we can only fit the
# foundpose results with a camera pose
for fres in foundposeests:
    imgpath = Path(fres["img_path"])
    name = imgpath.stem
    if name not in name2imgid.keys():
        name2imgid[name] = fres["img_id"]
    if name not in name2imgpath.keys():
        name2imgpath[name] = imgpath
    # could just use the "best" hypothesis. pretty often though, this hypothesis sucks.
    # valid_hyp = name in name2cam.keys() and fres["hypothesis_id"] == "0"
    valid_hyp = name in names
    if valid_hyp:
        camhyps.append(name2cam[name])
        if use_coarse:
            R = fres["R_coarse"]
            t = fres["t_coarse"]
        else:
            R = fres["R"]
            t = fres["t"]
        T = np.eye(4)
        T[:3, :3] = R
        T[:3, 3] = np.reshape(t, -1)
        obj2cams.append(v3d.Transform.from_matrix(T))
obj2cams = dca.stack(obj2cams)
camhyps = dca.stack(camhyps)

model, inlieridxs = ransac(
    camhyps.world_from_cam.matrix4x4,
    obj2cams.matrix4x4,
    fit_func=fitcams,
    loss_func=camloss,
    cost_func=camcost,
    samp_min=5,
    inlier_min=5,
    inlier_thres=0.15,
    max_iter=50,
    seed=0,
    relax_on_fail=True,
)

scalefactor = model.scale
camscaled = scale_cams(cams, scalefactor)
camhypsscaled = scale_cams(camhyps, scalefactor)
obj2worlds = camhypsscaled.world_from_cam @ obj2cams
obj2worldsinlier: v3d.Transform = obj2worlds[inlieridxs]

sceneptsscaled = scenepts.replace(p=scenepts.p * scalefactor)
segidxs = segment_pc_from_masks(sceneptsscaled, masks, camscaled, min_ratio=1/2)

# plane fit to seafloor
floormask = np.ones(len(sceneptsscaled), dtype=bool)
floormask[segidxs] = False
plane1 = pyrsc.Plane()
best_eq, best_inliers = plane1.fit(sceneptsscaled[floormask].p, thresh=0.005)
a, b, c, d = best_eq
normal = np.array([a, b, c])
R = get_axes_rot([0, 0, 1], normal)
planecent = [0, 0, -d / c]
Tmat = np.eye(4)
Tmat[:3, :3] = R
Tmat[:3, 3] = planecent
planeT = v3d.Transform.from_matrix(Tmat)
camzup = camscaled.apply_transform(planeT.inv)
if np.mean(camzup.world_from_cam.t[:, 2]) < 0:
    planeT = planeT @ v3d.Transform.from_matrix(np.array([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]]))
    camzup = camscaled.apply_transform(planeT.inv)
    normal = -normal

# 3x points from sfm point cloud
meshsamp, _ = trimesh.sample.sample_surface(objectmesh, count=len(segidxs))
objinscenepts = sceneptsscaled[segidxs]
# simple outlier removal to see if this works
o3dobj = o3d.geometry.PointCloud()
o3dobj.points = o3d.utility.Vector3dVector(objinscenepts.p)
_, inlieridx = o3dobj.remove_statistical_outlier(nb_neighbors=50, std_ratio=2.0)
tmpT = []
for i, obj2world in enumerate(obj2worldsinlier):
    samp_trf = obj2world @ meshsamp
    # source is sfm point cloud, since it is incomplete
    icpT = v3d.Transform.from_matrix(icp(objinscenepts.p[inlieridx], samp_trf, outlier_std=2.0))
    tmpT.append(icpT.inv @ obj2world)
    # debug figure (this takes a while to generate)
    meshpts = planeT.inv @ v3d.Point3d(p=samp_trf, rgb=[0, 0, 255])
    meshicppts = planeT.inv @ v3d.Point3d(p=icpT.inv @ samp_trf, rgb=[0, 255, 0])
    sceneobjzuppts = planeT.inv @ objinscenepts[inlieridx]
    sceneobjzupoutlierpts = planeT.inv @ objinscenepts[invert_idxs(inlieridx, len(objinscenepts))]
    sceneobjzupoutlierpts = sceneobjzupoutlierpts.replace(rgb=[255, 0, 0])
    xycent = np.mean(sceneobjzuppts.p, axis=0)[:2]
    centT = T_from_translation(-xycent[0], -xycent[1], 0)
    icpfig = v3d.make_fig(
        [centT @ meshpts, centT @ meshicppts, centT @ sceneobjzuppts, centT @ sceneobjzupoutlierpts],
    num_samples_point3d=1000)
    # icpfig.write_image(icpdebugdir / f"icpout_{str(i).zfill(4)}.png")
obj2worldsinlier = dca.stack(tmpT)

quatsinlier = quaternion.from_rotation_matrix(obj2worldsinlier.R)
ref = quatsinlier[0]
quatssymd = [ref]
for otherquat in quatsinlier[1:]:
    best = closest_quat_sym(ref, otherquat, objectsymmetries)
    quatssymd.append(best)
quatssymd = np.array(quatssymd)
obj2worldsinliersym = obj2worldsinlier.replace(R=quaternion.as_rotation_matrix(quatssymd))

qmeanransac, qinliers = ransac(quatssymd, fit_func=qmean, loss_func=qloss, cost_func=qcost, samp_min=5, inlier_min=5, inlier_thres=0.2, max_iter=50, relax_on_fail=True)

meanT = v3d.Transform(R=quaternion.as_rotation_matrix(qmeanransac), t=np.mean(obj2worldsinliersym.t, axis=0))
quatfig = v3d.make_fig(*get_axes_traces(obj2worldsinliersym, scale=0.5), *get_axes_traces(meanT, linewidth=10))

# burial ratio by fitting plane to floor point cloud
T_zup = planeT.inv @ meanT
meshzup = objectmesh.copy().apply_transform(T_zup.matrix4x4)
mesh_zvals = meshzup.vertices[:, 2]
zmin, zmax = np.min(mesh_zvals), np.max(mesh_zvals)
if zmin >= 0:
    burial_ratio_z = 0
else:
    burial_ratio_z = abs(zmin) / (abs(zmin) + zmax)
slicedmesh = trimesh.intersections.slice_mesh_plane(meshzup, [0, 0, 1], [0, 0, 0], cap=True)
burial_ratio_vol = 1 - slicedmesh.volume / objectmesh.volume
burial_depth = abs(zmin)

# visualization of fitted scene
meshzuppts = v3d.Point3d(p=meshzup.vertices)
scenezuppts = planeT.inv @ sceneptsscaled
aggfig = v3d.make_fig([scenezuppts, meshzuppts, camzup])

plane2camfit = camscaled.world_from_cam.inv @ planeT[..., None]
obj2camfit = camscaled.world_from_cam.inv @ meanT[..., None]
estposes = []
for name, obj2cam, plane2cam in zip(names, obj2camfit, plane2camfit):
    posedata = {
        "img_path": str(name2imgpath[name]),
        "img_id": name2imgid[name],
        "hypothesis_id": "0",
        "R": obj2cam.R.tolist(),
        "t": obj2cam.t.tolist(),
        "R_floor": plane2cam.R.tolist(),
        "t_floor": plane2cam.t.tolist(),
    }
    estposes.append(posedata)

In [ ]:
obj2world = obj2worldsinlier[2]
# samp_trf = obj2world @ meshpc
# meshpts = centT @ planeT.inv @ samp_trf.replace(rgb=[0, 0, 255])
# undo ICP transform
_, wireframetrace = get_trimesh_traces(mesh.copy().apply_transform((centT @ planeT.inv @ icpT @ obj2world).matrix4x4), wirecolor="rgb(0, 0, 255)")
# meshicppts = centT @ planeT.inv @ (icpT.inv @ samp_trf).replace(rgb=[0, 255, 0])
_, wireframeicptrace = get_trimesh_traces(mesh.copy().apply_transform((centT @ planeT.inv @ obj2world).matrix4x4), wirecolor="rgb(0, 255, 0)")
# sceneobjzuppts = planeT.inv @ objinscenepts[inlieridx]
sceneobjzuppts = planeT.inv @ objinscenepts
# sceneobjzupoutlierpts = planeT.inv @ objinscenepts[invert_idxs(inlieridx, len(objinscenepts))]
# sceneobjzupoutlierpts = centT @ sceneobjzupoutlierpts.replace(rgb=[181, 103, 0])
sceneobjzuppts = centT @ sceneobjzuppts.replace(rgb=[212, 130, 23])
xycent = np.mean(sceneobjzuppts.p, axis=0)[:2]
centT = T_from_translation(-xycent[0], -xycent[1], 0)
floorpts = centT @ planeT.inv @ sceneptsscaled[invert_idxs(segidxs, len(sceneptsscaled))]
floorpts = floorpts.replace(rgb=[255, 200, 200])
icpfig = v3d.make_fig(
    [wireframetrace, wireframeicptrace, sceneobjzuppts, sceneobjzupoutlierpts, floorpts],
num_samples_point3d=1000)
icpfig.update_layout(
    showlegend=False,
)
config = {
    "toImageButtonOptions": {
        "format": "png", # one of png, svg, jpeg, webp
        "filename": "icpvis",
        "width": None,
        "height": None,
        "scale": 1,
    }
}
icpfig.update_scenes(xaxis_visible=False, yaxis_visible=False, zaxis_visible=False)
icpfig.show(config=config)

In [ ]:
faintpc = centT @ planeT.inv @ sceneptsscaled
faintpc = faintpc.replace(rgb=[255, 200, 200])[::50]
trace = faintpc.make_traces()[0].update(marker={"color": "rgba(255, 10, 10, 0.3)", "size": 2})
newquatfig = v3d.make_fig(*get_axes_traces(centT @ planeT.inv @ obj2worldsinliersym, scale=0.5), *get_axes_traces(centT @ planeT.inv @ meanT, linewidth=10), trace)
# newquatfig = v3d.make_fig(*get_axes_traces(centT @ planeT.inv @ obj2worldsinliersym, scale=0.5), trace)
newquatfig.update_layout(
    showlegend=False,
)
config = {
    "toImageButtonOptions": {
        "format": "png", # one of png, svg, jpeg, webp
        "filename": "quatavgvis",
        "width": 1500,
        "height": 750,
        "scale": 1,
    }
}
newquatfig.update_scenes(xaxis_visible=False, yaxis_visible=False, zaxis_visible=False)
newquatfig.show(config=config)

In [ ]:
sceneobjzuppts = planeT.inv @ objinscenepts[inlieridx]
sceneobjzuppts = sceneobjzuppts.replace(rgb=[255, 0, 0])
xycent = np.mean(sceneobjzuppts.p, axis=0)[:2]
centT = T_from_translation(-xycent[0], -xycent[1], 0)
crap = centT @ planeT.inv @ sceneptsscaled[invert_idxs(segidxs, len(sceneptsscaled))]
crap = crap.replace(rgb=[255, 0, 0])
shit = centT @ planeT.inv @ sceneptsscaled[segidxs]
shit = shit.replace(rgb=[255, 200, 200])
xx, yy, zz = get_plane_zup(crap.p, n=10, z=0, square_grid=True)
planetrace = go.Surface(x=xx, y=yy, z=zz, opacity=0.5, colorscale="purples")
planeoutlinetrace = get_surface_line_traces(xx, yy, zz)
zupray = get_ray_trace([0, 1, 0], [0, 0, 1], length=1.5, width=5, color="#10c900")
trfmesh = mesh.copy().apply_transform((centT @ planeT.inv @ obj2world).matrix4x4)
facecols = ["#c98000"] * len(trfmesh.faces)
meshfig = ff.create_trisurf(x=trfmesh.vertices[:, 0], y=trfmesh.vertices[:, 1], z=trfmesh.vertices[:, 2], simplices=trfmesh.faces, color_func=facecols, show_colorbar=False)
planefitfig = v3d.make_fig(
    [crap, shit.make_traces()[0].update(marker={"color": "rgba(200, 1, 1, 0.005)"}), planetrace, *planeoutlinetrace, zupray, *meshfig.select_traces()],
num_samples_point3d=1000)
planefitfig.update_layout(
    showlegend=False,
)
# 1200/450
config = {
    "toImageButtonOptions": {
        "format": "png", # one of png, svg, jpeg, webp
        "filename": "planefitvis",
        "width": 2400,
        "height": 900,
        "scale": 1,
    }
}
planefitfig.update_scenes(xaxis_visible=False, yaxis_visible=False, zaxis_visible=False)
planefitfig.show(config=config)

## map of burial fractions

In [ ]:
multiview_fitted = True
pose_type = "refined"
use_icp = True
metricsfiltdf = metricsdf[(metricsdf["multiview_fitted"] == multiview_fitted) & (metricsdf["pose_type"] == pose_type) & (metricsdf["use_icp"] == use_icp)]
burialdepths = metricsfiltdf["burial_depth"].values
burialdepths_gt = metricsfiltdf["burial_depth_gt"].values
burialdepths_err = metricsfiltdf["burial_error_depth"].values
lats = metricsfiltdf["lat"].values
lons = metricsfiltdf["lon"].values

def within_domain(lats, lons, domain):
    latmin, latmax = domain["S"], domain["N"]
    lonmin, lonmax = domain["W"], domain["E"]
    return (latmin <= lats) & (lats <= latmax) & (lonmin <= lons) & (lons <= lonmax)

### newer figures

#### pcolor plots

In [ ]:
from sklearn.cluster import KMeans, SpectralClustering, MeanShift, AgglomerativeClustering

positions = np.array([lons, lats]).T
# clustermodel = KMeans(n_clusters=5, random_state=0).fit(positions)
# clustermodel = SpectralClustering(n_clusters=5, random_state=0).fit(positions)
# clustermodel = MeanShift(bandwidth=10).fit(positions)
clustermodel = AgglomerativeClustering(n_clusters=None, distance_threshold=0.007).fit(positions)
posclass = clustermodel.labels_
plt.scatter(lons, lats, c=posclass)

In [ ]:
domain = generate_domain(lats, lons, padding=0.01)
xi = np.linspace(domain["W"], domain["E"], 500)
yi = np.linspace(domain["S"], domain["N"], 500)
grid = np.meshgrid(xi, yi)
fig, ax = get_carree_axis(domain=domain, land=True)
ax.add_feature(cartopy.feature.OCEAN, facecolor="#dbf7ff")
get_carree_gl(ax)
for i in range(clustermodel.n_clusters_):
    mask = posclass == i
    if np.sum(mask) <= 2:
        continue
    zi = scipy.interpolate.griddata(np.array([lons, lats]).T[mask], burialdepths_gt[mask], grid, method="linear")
    pcmesh = ax.pcolormesh(xi, yi, zi)
# pcmesh = ax.pcolormesh(xi, yi, zi)
# sc = ax.scatter(lons, lats, color="red", edgecolor="k", s=4)
cb = plt.colorbar(pcmesh, label="Burial Depths (m)")
plt.show()

In [ ]:
# domain = generate_domain(lats, lons, padding=0.3)
domain = {
    "W": -118.6,
    "E": -118.2,
    "S": 33.33,
    "N": 33.62,
}
# insetdomain = generate_domain(lats, lons, padding=0.01)
insetdomain = {
    "W": -118.445,
    "E": -118.415,
    "S": 33.559,
    "N": 33.568,
}
vmin = min(np.min(burialdepths_gt), np.min(burialdepths))
vmax = max(np.max(burialdepths_gt), np.max(burialdepths))
xi = np.linspace(domain["W"], domain["E"], 1000)
yi = np.linspace(domain["S"], domain["N"], 1000)
grid = np.meshgrid(xi, yi)
zi = scipy.interpolate.griddata(np.array([lons, lats]).T, burialdepths_gt, grid, method="linear")
fig, ax = get_carree_axis(domain=domain, land=True)
ax.add_feature(cartopy.feature.OCEAN, facecolor="#dbf7ff")
get_carree_gl(ax)
for i in range(clustermodel.n_clusters_):
    mask = posclass == i
    if np.sum(mask) < 2:
        continue
    zi = scipy.interpolate.griddata(np.array([lons, lats]).T[mask], burialdepths_gt[mask], grid, method="linear")
    pcmesh = ax.pcolormesh(xi, yi, zi, vmin=vmin, vmax=vmax)
cb = plt.colorbar(pcmesh, label="Burial Depth (m)")
sc = ax.scatter(lons, lats, color="red", edgecolor="k", s=5, alpha=0.7)
axins = ax.inset_axes(
    [0.1, 0.05, 0.87, 0.68],
    xlim=(insetdomain["W"], insetdomain["E"]),
    ylim=(insetdomain["S"], insetdomain["N"]),
    xticklabels=[], yticklabels=[]
)
xi = np.linspace(insetdomain["W"], insetdomain["E"], 100)
yi = np.linspace(insetdomain["S"], insetdomain["N"], 100)
grid = np.meshgrid(xi, yi)
for i in range(clustermodel.n_clusters_):
    mask = posclass == i
    if np.sum(mask) <= 2:
        continue
    zi = scipy.interpolate.griddata(np.array([lons, lats]).T[mask], burialdepths_gt[mask], grid, method="linear")
    pcmesh = axins.pcolormesh(xi, yi, zi, vmin=vmin, vmax=vmax, edgecolors="face")
axins.scatter(lons, lats, color="red", edgecolor="k", s=3, alpha=0.7)
axins.set_facecolor("#dbf7ff")
ax.indicate_inset_zoom(axins, edgecolor="black")
fig.suptitle("Absolute burial depth GT")
# fig.savefig(tmpdir / "burialmapgt_pcolor.png", dpi=300, bbox_inches="tight")
# plt.show()

#### scatter plots

In [ ]:
cmap = "plasma"
# domain = generate_domain(lats, lons, padding=0.3)
domain = {
    "W": -118.6,
    "E": -118.2,
    "S": 33.42,
    "N": 33.62,
}
# insetdomain = generate_domain(lats, lons, padding=0.01)
insetdomain = {
    "W": -118.445,
    "E": -118.415,
    "S": 33.559,
    "N": 33.568,
}
vmin = min(np.min(burialdepths_gt), np.min(burialdepths))
vmax = max(np.max(burialdepths_gt), np.max(burialdepths))
fig, ax = get_carree_axis(domain=domain, land=True)
ax.add_feature(cartopy.feature.OCEAN, facecolor="#dbf7ff")
get_carree_gl(ax)
sc = ax.scatter(lons, lats, c=burialdepths_gt, s=3, alpha=0.8, vmin=vmin, vmax=vmax, cmap=cmap)
axins = ax.inset_axes(
    [0.1, 0.0, 0.87, 0.68],
    projection=ccrs.PlateCarree(),
    xlim=(insetdomain["W"], insetdomain["E"]),
    ylim=(insetdomain["S"], insetdomain["N"]),
    xticklabels=[], yticklabels=[]
)
get_carree_gl(axins, labels=False)
scin = axins.scatter(lons, lats, c=burialdepths_gt, s=5, alpha=0.8, vmin=vmin, vmax=vmax, cmap=cmap)
cb = plt.colorbar(sc, ax=ax, label="Burial Depth (m)", fraction=0.025, pad=0.04)
axins.set_facecolor("#dbf7ff")
ax.indicate_inset_zoom(axins, edgecolor="black")
# fig.suptitle("Absolute burial depth GT")
fig.savefig(tmpdir / "burialmapgt.png", dpi=300, bbox_inches="tight")
# plt.savefig(tmpdir / "burialmapgt.png", dpi=300, bbox_inches="tight")
# plt.show()

In [ ]:
fig, ax = get_carree_axis(domain=insetdomain, land=True)
ax.add_feature(cartopy.feature.OCEAN, facecolor="#dbf7ff")
get_carree_gl(ax)
sc = ax.scatter(lons, lats, c=burialdepths, s=5, alpha=0.8, vmin=vmin, vmax=vmax, cmap=cmap)
cb = plt.colorbar(scin, ax=ax, label="Burial Depth (m)", fraction=0.025, pad=0.04)
# fig.suptitle("Absolute burial depth predicted")
fig.savefig(tmpdir / "burialmappred.png", dpi=300, bbox_inches="tight")

In [ ]:
errmin = np.min(burialdepths_err)
errmax = np.max(burialdepths_err)
fig, ax = get_carree_axis(domain=insetdomain, land=True)
ax.add_feature(cartopy.feature.OCEAN, facecolor="#dbf7ff")
get_carree_gl(ax)
sc = ax.scatter(lons, lats, c=burialdepths_err, s=5, alpha=0.8, vmin=errmin, vmax=errmax, cmap=cmap)
cb = plt.colorbar(scin, ax=ax, label="Absolute Burial Depth Error (m)", fraction=0.025, pad=0.04)
# fig.suptitle("Absolute burial depth predicted")
fig.savefig(tmpdir / "burialmaperr.png", dpi=300, bbox_inches="tight")

In [ ]:
lookhere = filtermetrics(metricsdf, name="dive3-depthcharge-05-11", multiview_fitted=True, use_icp=True, pose_type="coarse").iloc[0]
# lookhere = filtermetrics(metricsdf, name="barrel2", multiview_fitted=True, use_icp=True, pose_type="coarse").iloc[0]
# lookhere = filtermetrics(metricsdf, name="dive6-smokefloat-10-17-26", multiview_fitted=True, use_icp=True, pose_type="coarse").iloc[0]
trace = go.Scatter(
    x=lons,
    y=lats,
    mode="markers",
    marker=dict(
        size=5,
    ),
)
tracesingle = go.Scatter(
    x=[lookhere["lon"]],
    y=[lookhere["lat"]],
    mode="markers",
    marker=dict(
        size=10,
    ),
)
go.Figure(data=[trace, tracesingle]).show()